#### Step 1: Item Reduction

Which survey items are actually measuring distinct, meaningful constructs vs. which ones are redundant, unclear or not contributing much?

We want to keep the questions that clearly measure different underlying social constructs. We should do this for both the implicit and the explicit variables, as we hypothesize that both are driven by similar underlying psychological profiles.

Does our data require item reduction? Should we perform any theoretical selection? Filtering before our factor analysis.

We might firstly check the Kaiser-Meyer-Olkin (KMR) measure of sampling adequacy. This is essentially of measure of how correlated our variables are. We are looking for a high value close to 1 which indicates high correlation between related variables, but not too much multicollinearity, and an effective sample size.  

In [ ]:
# KMO Analysis
import pandas as pd
from factor_analyzer import calculate_kmo

data = pd.read_pickle("../data/survey_data_cleaned.pkl")

def get_kmo(variables, name):
    df_clean = data[variables].dropna()
    kmo_all, kmo_model = calculate_kmo(df_clean.values.astype(float))
    print(f"{name}: KMO = {kmo_model:.3f}")
    return kmo_model

# Variable groups
emotional_vars = [col for col in data.columns if col.startswith(('es_', 'el_'))]
workplace_vars = [col for col in data.columns if col.startswith(('wfs_', 'wfl_'))]
latent_vars = [col for col in data.columns if col.startswith(('es_', 'wfs_'))]
explicit_vars = [col for col in data.columns if col.startswith(('el_', 'wfl_'))]

# Calculate KMO
print("KMO Results:")
get_kmo(emotional_vars, "Emotional")
get_kmo(workplace_vars, "Workplace") 
get_kmo(latent_vars, "Latent (Fast Choice)")
get_kmo(explicit_vars, "Explicit (Likert)")

Given these fairly excellent KMO results, we don't want to reduce our items at all, as this may reduce meaningful information provide by our excellent survey results. Hence, we're ready for factor analysis.

#### Step 2: Extraction of Factors

Here we'll select factor variables, standardize our data, apply kaiser criterion, create scree plots and extract final factors.

But firstly, we should just look at what the data can tell us. 

Let's look at the correlation matrix for our two categories of responses: fast-choice and likert. 

In [2]:
# Create informative variable name mappings from codebook with symbols
emotional_names = {
    'es_fgen': '* Ready for next step',
    'es_ftra_des': '* Want new start', 
    'es_ftra_aut': '* Freedom to change',
    'es_fcnt_com': '* Comfortable where I am',
    'es_fcnt_psy': '* Voice is heard',
    'es_fcon_soc': '* Others influence decisions',
    'es_fbal_anx': '* Anxious about change',
    'es_fres_fin': '* Financially motivated',
    'es_fcont_imp': '* Believe in myself',
    'es_fjou_opt': '* Optimistic about future',
    'es_fcon_inc': '* Feel included',
    'es_fbal_sat': '* Happy where I am',
    'es_fres_ski': '* Skills to progress',
    'es_fcont_con': '* Control next step',
    'es_fjou_pro': '* Set myself goals',
    'el_lgen': '* Ready for next step (L)',
    'el_ltra_des': '* Want new start (L)',
    'el_ltra_aut': '* Freedom to change (L)', 
    'el_lcnt_com': '* Comfortable where I am (L)',
    'el_lcnt_psy': '* Voice is heard (L)',
    'el_lcon_soc': '* Others influence decisions (L)',
    'el_lbal_anx': '* Anxious about change (L)',
    'el_lres_fin': '* Financially motivated (L)',
    'el_lcont_imp': '* Believe in myself (L)',
    'el_ljou_opt': '* Optimistic about future (L)',
    'el_lcon_inc': '* Feel included (L)',
    'el_lbal_sat': '* Happy where I am (L)',
    'el_lres_ski': '* Skills to progress (L)',
    'el_lcont_con': '* Control next step (L)',
    'el_ljou_pro': '* Set myself goals (L)'
}

workplace_names = {
    'wfs_fuse_skills': '◆ Can use skills',
    'wfs_flearn_dev': '◆ Learning opportunities',
    'wfs_fcarprom': '◆ Can grow here',
    'wfs_fmean_full': '◆ Job meaningful',
    'wfs_fpoorman': '◆ Manager is poor',
    'wfs_ftoxic': '◆ Culture is toxic',
    'wfs_fexcess_wk': '◆ Working too hard',
    'wfs_fcollea': '◆ Don\'t get along with colleagues',
    'wfs_fwellcomp': '◆ Well compensated',
    'wfs_ffin_fair': '◆ Salary unfair vs colleagues',
    'wfs_fenjhyb': '◆ Enjoy hybrid working',
    'wfl_luse_skills': '◆ Can use skills (L)',
    'wfl_llearn_dev': '◆ Learning opportunities (L)',
    'wfl_lcarprom_d': '◆ Can grow here (L)',
    'wfl_lmean_full': '◆ Job meaningful (L)',
    'wfl_lpoorman': '◆ Manager is poor (L)',
    'wfl_ltoxic': '◆ Culture is toxic (L)',
    'wfl_lexcess_wk': '◆ Working too hard (L)',
    'wfl_lcollea': '◆ Don\'t get along with colleagues (L)',
    'wfl_lwellcomp': '◆ Well compensated (L)',
    'wfl_lfin_lair': '◆ Salary unfair vs colleagues (L)',
    'wfl_lenjhyb': '◆ Enjoy hybrid working (L)'
}

# Get variable lists
fast_choice_vars = [col for col in data.columns if col.startswith(('es_', 'wfs_'))]
likert_vars = [col for col in data.columns if col.startswith(('el_', 'wfl_'))]

# Calculate correlations
fast_choice_corr = data[fast_choice_vars].corr()
likert_corr = data[likert_vars].corr()

# Create combined name mapping
all_names = {**emotional_names, **workplace_names}

# Rename correlation matrix indices and columns
fast_choice_corr_renamed = fast_choice_corr.rename(index=all_names, columns=all_names)
likert_corr_renamed = likert_corr.rename(index=all_names, columns=all_names)

import seaborn as sns
import matplotlib.pyplot as plt

# Create color coding for variable types
def add_variable_type_colors(ax, var_names, var_list):
    """Add colored bars to distinguish emotional vs workplace variables"""
    colors = []
    for var in var_list:
        if var.startswith(('es_', 'el_')):
            colors.append('#FF6B6B')  # Red for emotional
        else:
            colors.append('#4ECDC4')  # Teal for workplace
    
    # Add colored bar on left side
    for i, color in enumerate(colors):
        ax.add_patch(plt.Rectangle((-0.1, i), 0.1, 1, color=color, clip_on=False))
    
    # Add colored bar on bottom
    for i, color in enumerate(colors):
        ax.add_patch(plt.Rectangle((i, -0.1), 1, 0.1, color=color, clip_on=False))

# Fast choice correlation matrix
fig, ax = plt.subplots(figsize=(14, 10))
sns.heatmap(fast_choice_corr_renamed, annot=False, cmap="coolwarm", square=True, 
           cbar_kws={"shrink": .8}, vmin=-1, vmax=1, ax=ax)
ax.set_title("Fast Choice Variables Correlation Matrix", fontsize=16, pad=20)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
ax.set_yticklabels(ax.get_yticklabels(), rotation=0)

# Add legend
legend_elements = [plt.Line2D([0], [0], marker='*', color='w', markerfacecolor='black', 
                             markersize=10, label='* Emotional Variables'),
                  plt.Line2D([0], [0], marker='D', color='w', markerfacecolor='black', 
                            markersize=8, label='◆ Workplace Variables')]
ax.legend(handles=legend_elements, loc='upper left', bbox_to_anchor=(1.05, 1))

plt.tight_layout()
plt.show()

# Likert correlation matrix  
fig, ax = plt.subplots(figsize=(14, 10))
sns.heatmap(likert_corr_renamed, annot=False, cmap="coolwarm", square=True,
           cbar_kws={"shrink": .8}, vmin=-1, vmax=1, ax=ax)
ax.set_title("Likert Variables Correlation Matrix", fontsize=16, pad=20)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
ax.set_yticklabels(ax.get_yticklabels(), rotation=0)

# Add legend
legend_elements = [plt.Line2D([0], [0], marker='*', color='w', markerfacecolor='black', 
                             markersize=10, label='* Emotional Variables'),
                  plt.Line2D([0], [0], marker='D', color='w', markerfacecolor='black', 
                            markersize=8, label='◆ Workplace Variables')]
ax.legend(handles=legend_elements, loc='upper left', bbox_to_anchor=(1.05, 1))

plt.tight_layout()
plt.show()

NameError: name 'data' is not defined

Immediately we can see some very clear patterns.

Fast Choice Vars:
- Correlations are not as strong comparatively
- Covariance among subset of emotional variables, specifically those related to believing in one's self and future planning
- Some very logical correlations, between 'can grow here' and 'learning opportunities/job meaningful'
- Some clear groupings, it would make sense that there's latent factors explaining our variance
- Emotional vars tend to co-vary, as do Workplace/Functional, but covariance between the two is less common 

Likert Variables:
- Stronger correlations in general, especially within the emotional/workplace groupings
- A lot less gray, implies that likert responses are just more extreme (i.e. perhaps people are more decisive when they have time to think about their answers)

Interestingly, in both, some questions just don't correlate much at all.

What about correlations with "Ready for next step"

Fast choice:
- Positive: Believe in myself, In control of next step, freedom to change etc.
- Negative: Anxious about change, others influence financial decisions, 

Likert:
- Positive: Set myself goals, freedom to change, skills to progress, can use skills
- Negative: Anxious about change, others influence decisions, don't get along with colleagues, salary unfair, manager poor/toxic 

We can really start to see why a factor analysis is beneficial for our data. There's a lot of variables and variance patterns to sort through. A manually encoding approach would probably introduce a lot of measurement error. Likewise, it's very plausible that our factor analysis would pick up a real 'latent factors'.

##### Now let's proceed onto the factor analysis.

Need to evaluate:
1) factor loading - how high the correlation between the variable and the factor
2) eigenvalue - how much variance can be explained by the factor of all variables
3) communality - how much variance of the total variables can be explained by the factors 

To do the above, we need the communalities and the factor component matrix.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import FactorAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from scipy.stats import bartlett
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity, calculate_kmo

# Variable name mappings from your code
emotional_names = {
    'es_fgen': '* Ready for next step',
    'es_ftra_des': '* Want new start', 
    'es_ftra_aut': '* Freedom to change',
    'es_fcnt_com': '* Comfortable where I am',
    'es_fcnt_psy': '* Voice is heard',
    'es_fcon_soc': '* Others influence decisions',
    'es_fbal_anx': '* Anxious about change',
    'es_fres_fin': '* Financially motivated',
    'es_fcont_imp': '* Believe in myself',
    'es_fjou_opt': '* Optimistic about future',
    'es_fcon_inc': '* Feel included',
    'es_fbal_sat': '* Happy where I am',
    'es_fres_ski': '* Skills to progress',
    'es_fcont_con': '* Control next step',
    'es_fjou_pro': '* Set myself goals',
    'el_lgen': '* Ready for next step (L)',
    'el_ltra_des': '* Want new start (L)',
    'el_ltra_aut': '* Freedom to change (L)', 
    'el_lcnt_com': '* Comfortable where I am (L)',
    'el_lcnt_psy': '* Voice is heard (L)',
    'el_lcon_soc': '* Others influence decisions (L)',
    'el_lbal_anx': '* Anxious about change (L)',
    'el_lres_fin': '* Financially motivated (L)',
    'el_lcont_imp': '* Believe in myself (L)',
    'el_ljou_opt': '* Optimistic about future (L)',
    'el_lcon_inc': '* Feel included (L)',
    'el_lbal_sat': '* Happy where I am (L)',
    'el_lres_ski': '* Skills to progress (L)',
    'el_lcont_con': '* Control next step (L)',
    'el_ljou_pro': '* Set myself goals (L)'
}

workplace_names = {
    'wfs_fuse_skills': '◆ Can use skills',
    'wfs_flearn_dev': '◆ Learning opportunities',
    'wfs_fcarprom': '◆ Can grow here',
    'wfs_fmean_full': '◆ Job meaningful',
    'wfs_fpoorman': '◆ Manager is poor',
    'wfs_ftoxic': '◆ Culture is toxic',
    'wfs_fexcess_wk': '◆ Working too hard',
    'wfs_fcollea': '◆ Don\'t get along with colleagues',
    'wfs_fwellcomp': '◆ Well compensated',
    'wfs_ffin_fair': '◆ Salary unfair vs colleagues',
    'wfs_fenjhyb': '◆ Enjoy hybrid working',
    'wfl_luse_skills': '◆ Can use skills (L)',
    'wfl_llearn_dev': '◆ Learning opportunities (L)',
    'wfl_lcarprom_d': '◆ Can grow here (L)',
    'wfl_lmean_full': '◆ Job meaningful (L)',
    'wfl_lpoorman': '◆ Manager is poor (L)',
    'wfl_ltoxic': '◆ Culture is toxic (L)',
    'wfl_lexcess_wk': '◆ Working too hard (L)',
    'wfl_lcollea': '◆ Don\'t get along with colleagues (L)',
    'wfl_lwellcomp': '◆ Well compensated (L)',
    'wfl_lfin_lair': '◆ Salary unfair vs colleagues (L)',
    'wfl_lenjhyb': '◆ Enjoy hybrid working (L)'
}

# Combine mappings
all_names = {**emotional_names, **workplace_names}

# Get variable lists from your existing code
fast_choice_vars = [col for col in data.columns if col.startswith(('es_', 'wfs_'))]
likert_vars = [col for col in data.columns if col.startswith(('el_', 'wfl_'))]

print(f"Fast choice variables: {len(fast_choice_vars)}")
print(f"Likert variables: {len(likert_vars)}")

def perform_factor_analysis(df, var_list, title_prefix):
    """Perform factor analysis with scree plot"""
    
    # Select and clean data
    fa_data = df[var_list].dropna()
    print(f"\n{title_prefix} - Sample size after removing missing: {len(fa_data)}")
    
    # Standardize data
    scaler = StandardScaler()
    fa_data_scaled = scaler.fit_transform(fa_data)
    fa_data_scaled = pd.DataFrame(fa_data_scaled, columns=var_list)
    
    # Test assumptions
    # Bartlett's test for sphericity
    chi_square_value, p_value = calculate_bartlett_sphericity(fa_data_scaled)
    print(f"Bartlett's test: Chi-square = {chi_square_value:.2f}, p-value = {p_value:.4f}")
    
    # Kaiser-Meyer-Olkin test
    kmo_all, kmo_model = calculate_kmo(fa_data_scaled)
    print(f"KMO test: {kmo_model:.3f}")
    
    # Scree plot - determine number of factors
    fa_scree = FactorAnalyzer(n_factors=len(var_list), rotation=None)
    fa_scree.fit(fa_data_scaled)
    
    # Get eigenvalues
    eigenvalues, _ = fa_scree.get_eigenvalues()
    
    # Create scree plot
    plt.figure(figsize=(10, 6))
    factors = range(1, len(eigenvalues) + 1)
    plt.plot(factors, eigenvalues, 'bo-', linewidth=2, markersize=8)
    plt.axhline(y=1, color='r', linestyle='--', alpha=0.7, label='Kaiser criterion (eigenvalue = 1)')
    plt.title(f'{title_prefix} - Scree Plot')
    plt.xlabel('Factor Number')
    plt.ylabel('Eigenvalue')
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    # Add eigenvalue labels
    for i, ev in enumerate(eigenvalues[:10]):  # Label first 10
        plt.annotate(f'{ev:.2f}', (i+1, ev), textcoords="offset points", 
                    xytext=(0,10), ha='center', fontsize=9)
    
    plt.tight_layout()
    plt.show()
    
    # Determine number of factors (Kaiser criterion: eigenvalue > 1)
    n_factors = sum(eigenvalues > 1)
    print(f"Number of factors (Kaiser criterion): {n_factors}")
    
    # Alternative: explained variance criterion (e.g., 60-70%)
    cumvar = np.cumsum(eigenvalues) / sum(eigenvalues) * 100
    n_factors_60 = sum(cumvar < 60) + 1
    n_factors_70 = sum(cumvar < 70) + 1
    
    print(f"Factors explaining 60% variance: {n_factors_60}")
    print(f"Factors explaining 70% variance: {n_factors_70}")
    print(f"First 5 factors explain: {cumvar[4]:.1f}% of variance")
    
    # Perform factor analysis with determined number of factors
    n_factors_final = max(2, min(n_factors, 5))  # Use 2-5 factors
    print(f"\nUsing {n_factors_final} factors for analysis")
    
    # Factor analysis with varimax rotation
    fa = FactorAnalyzer(n_factors=n_factors_final, rotation='varimax')
    fa.fit(fa_data_scaled)
    
    # Get results with proper variable names
    loadings = pd.DataFrame(fa.loadings_, 
                           columns=[f'Factor{i+1}' for i in range(n_factors_final)],
                           index=var_list)
    
    # Add descriptive names
    loadings['Variable_Name'] = loadings.index.map(all_names)
    loadings = loadings[['Variable_Name'] + [f'Factor{i+1}' for i in range(n_factors_final)]]
    
    communalities = pd.DataFrame({'Variable_Name': [all_names.get(var, var) for var in var_list],
                                 'Communality': fa.get_communalities()}, 
                                index=var_list)
    
    eigenvalues_final = fa.get_eigenvalues()[0][:n_factors_final]
    
    # Print results
    print(f"\n=== {title_prefix} RESULTS ===")
    print(f"\nEigenvalues and Variance Explained:")
    total_variance = sum(eigenvalues_final)
    cumulative = 0
    for i, ev in enumerate(eigenvalues_final):
        var_explained = ev / len(var_list) * 100
        prop_explained = ev / total_variance * 100
        cumulative += prop_explained
        print(f"Factor {i+1}: {ev:.3f} | {var_explained:.1f}% of total | {prop_explained:.1f}% of extracted | {cumulative:.1f}% cumulative")
    
    # Factor loadings heatmap
    print(f"\nFactor Loadings Matrix:")
    
    # Prepare data for heatmap
    loadings_heatmap = loadings.copy()
    factor_cols = [f'Factor{i+1}' for i in range(n_factors_final)]
    loadings_matrix = loadings_heatmap[factor_cols]
    
    # Create heatmap
    plt.figure(figsize=(8, max(6, len(var_list) * 0.3)))
    
    # Use RdBu_r colormap (red-white-blue reversed) for better positive/negative distinction
    sns.heatmap(loadings_matrix, 
                annot=True, 
                fmt='.3f',
                cmap='RdBu_r', 
                center=0,
                vmin=-1, 
                vmax=1,
                cbar_kws={'label': 'Factor Loading'},
                yticklabels=[all_names.get(var, var) for var in var_list])
    
    plt.title(f'{title_prefix} - Factor Loadings Matrix')
    plt.xlabel('Factors')
    plt.ylabel('Variables')
    plt.tight_layout()
    plt.show()
    
    # Communalities table
    print(f"\nCommunalities (sorted by value):")
    print("=" * 50)
    comm_sorted = communalities.sort_values('Communality', ascending=False)
    print(f"{'Variable':<35} {'Communality':>10}")
    print("-" * 47)
    for idx, row in comm_sorted.iterrows():
        var_name = row['Variable_Name'][:33] + ('...' if len(row['Variable_Name']) > 33 else '')
        comm_val = row['Communality']
        print(f"{var_name:<35} {comm_val:>8.3f}")
    
    # Low communalities warning
    low_comm = communalities[communalities['Communality'] < 0.3]
    if len(low_comm) > 0:
        print(f"\n⚠️  Variables with low communalities (< 0.3) - consider removal:")
        for idx, row in low_comm.iterrows():
            print(f"   {row['Variable_Name']}: {row['Communality']:.3f}")
    else:
        print(f"\n✅ All variables have adequate communalities (≥ 0.3)")
    
    return {
        'loadings': loadings,
        'communalities': communalities,
        'eigenvalues': eigenvalues_final,
        'factor_analyzer': fa,
        'n_factors': n_factors_final
    }

# Perform factor analysis for both groups
print("="*60)
print("FACTOR ANALYSIS")
print("="*60)

# Fast choice variables
results_fast = perform_factor_analysis(data, fast_choice_vars, "Fast Choice Variables")

# Likert variables  
results_likert = perform_factor_analysis(data, likert_vars, "Likert Variables")


##### Interpreting the factors

Factor 1: Appears to capture career agency/self-efficacy - high loadings on "Ready for next step" (0.687), "Set myself goals" (0.606), "Control next step" (0.572), "Believe in myself" (0.563)

Factor 2: Looks like workplace satisfaction/growth - strong loadings on "Can grow here" (0.757), "Job meaningful" (0.711), "Learning opportunities" (0.675), "Can use skills" (0.650)

Factor 3: Represents workplace dysfunction - high loadings on "Culture is toxic" (0.643), "Manager is poor" (0.681), "Salary unfair vs colleagues" (0.595)

Factor 4: Mixed factor, harder to interpret cleanly (probably left-over variance, non correlated variables)

Factor 5: Also mixed, potentially workplace comfort but weak loadings (probably left-over variance)